Camille DUPRE LA TOUR & Maha EL ALLEM DIA2

In [4]:
import csv

In [5]:
ROLL_LENGTH = 500

In [6]:
# Initialize the dough roll (0 = no defect, otherwise defect counts for each class)
roll = [{'a': 0, 'b': 0, 'c': 0, 'occupied': False} for _ in range(ROLL_LENGTH)]

# Load defect data from the CSV file
def load_defects(file_path):
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            position = round(float(row['x']))  # Convert 'x' to nearest integer
            defect_class = row['class']        # Read the defect class ('a', 'b', 'c')

            if 0 <= position < ROLL_LENGTH:  # Ensure position is within bounds
                roll[position][defect_class] += 1

# Call the function to load the defects
load_defects('defects.csv')

# Represent biscuits with their properties
biscuits = [
    {'type': 0, 'size': 4, 'value': 3, 'tolerances': {'a': 4, 'b': 2, 'c': 3}},
    {'type': 1, 'size': 8, 'value': 12, 'tolerances': {'a': 5, 'b': 4, 'c': 4}},
    {'type': 2, 'size': 2, 'value': 1, 'tolerances': {'a': 1, 'b': 2, 'c': 1}},
    {'type': 3, 'size': 5, 'value': 8, 'tolerances': {'a': 2, 'b': 3, 'c': 2}}
]


This code initializes a simulation for placing biscuits on a roll of dough while considering defect data loaded from a CSV file.

1. **Roll Initialization:**  
   A list named `roll` is created to represent the dough, consisting of 500 segments (`ROLL_LENGTH`). Each segment is initialized with zero defects for three defect types (`a`, `b`, `c`) and marked as unoccupied (`occupied = False`).

2. **Defect Loading:**  
   The `load_defects` function reads defect data from a CSV file (`defects.csv`). Each row contains the position (`x`) and defect class (`a`, `b`, or `c`). The position is rounded to the nearest integer to align with the roll segments. If the position falls within valid bounds (0 to 499), the corresponding defect count for that segment is incremented.

3. **Biscuits Representation:**  
   The `biscuits` list defines four types of biscuits, each described by:  
   - `type`: Identifier for the biscuit type.  
   - `size`: Number of segments the biscuit occupies on the roll.  
   - `value`: A score representing the biscuit's importance or utility.  
   - `tolerances`: Maximum acceptable defect counts for each defect type (`a`, `b`, `c`).

This setup provides the foundation for simulating biscuit placement on the dough roll, balancing defect constraints and biscuit properties.

In [7]:
def can_place_biscuit(position, biscuit, roll):
    """
    Checks if a biscuit can be placed at the given position on the roll,
    considering both the occupation of positions and the defect tolerance of the biscuit.

    Parameters:
    position (int): The starting position where the biscuit will be placed.
    biscuit (dict): The biscuit to be placed, containing its size, defect tolerance, and defect information.
    roll (list): The dough roll with defect counts and 'occupied' flags.

    Returns:
    bool: True if the biscuit can be placed, False otherwise.
    """
    total_defects = {'a': 0, 'b': 0, 'c': 0}  # Track total defects within the area
    for i in range(biscuit['size']):
        occupied_position = position + i

        # Check if the position is within bounds and not already occupied
        if  occupied_position >= ROLL_LENGTH or roll[occupied_position]['occupied']:
            return False  # Position is either out of bounds or already occupied

        # Add defects at the current position to total defects
        total_defects['a'] += roll[occupied_position]['a']
        total_defects['b'] += roll[occupied_position]['b']
        total_defects['c'] += roll[occupied_position]['c']

    # Check if the total defects exceed the biscuit's tolerance
    for defect_type in ['a', 'b', 'c']:
        if total_defects[defect_type] > biscuit['tolerances'][defect_type]:
            return False  # Defects exceed tolerance for this type of defect

    return True  # All checks passed, biscuit can be placed




The placement function ensures that biscuits are placed validly on the dough roll. It checks that the target position is unoccupied and verifies the defect constraints for the specified biscuit. This ensures no overlapping and compliance with defect tolerances.

If the biscuit can be placed (as determined by the `can_place_biscuit` function), then the following function will handle the placement of the biscuit on the dough roll and update the defect counts accordingly.

In [8]:
def place_biscuit(start_position, biscuit, roll):
    """
    Places a biscuit on the dough roll at the given starting position,

    Parameters:
    start_position (int): The starting position where the biscuit will be placed.
    biscuit (dict): The biscuit to be placed, containing its size, defect counts, and defect tolerance.
    roll (list): The dough roll with defect counts and 'occupied' flags.
    """
    # Mark the positions as occupied and update defect counts
    for i in range(biscuit['size']):
        position = start_position + i

        # Mark the position as occupied
        roll[position]['occupied'] = True

In [9]:
def calculate_unused_penalty(roll):
    """
    Calculates the penalty for unused dough on the roll.

    Parameters:
    roll (list of dict): The dough roll with defect counts.

    Returns:
    int: Total penalty for unused dough.
    """
    covered = set()

    # Mark positions covered by biscuits (positions with defect counts greater than 0)
    for i, position in enumerate(roll):
        if any(count > 0 for count in position.values()):  # If any defect count is > 0, position is covered
            covered.add(i)

    # Uncovered positions are those not in covered
    unused_count = len([i for i in range(len(roll)) if i not in covered])

    # Penalty is -1 per unused unit
    return -unused_count


This function calculates the penalty for unused dough on the roll. It identifies positions that are not covered by biscuits (positions with no defect counts) and calculates the penalty based on the number of uncovered positions. Each unused position contributes a penalty of -1. The roll is checked to see which positions have biscuits placed, based on the presence of defects, and the unused penalty is then calculated accordingly.

In [10]:
def greedy_biscuit_placement(roll, biscuits):
    # Initialize counters for each type of biscuit
    biscuit_counts = {0: 0, 1: 0, 2: 0, 3: 0}  # Counter for each biscuit type
    total_value = 0  # Total value of placed biscuits

    # Sort biscuits by value-to-size ratio (greedy heuristic)
    biscuits = sorted(biscuits, key=lambda biscuit: biscuit['value'] / biscuit['size'], reverse=True)

    # Iterate through each biscuit starting with the one with the highest value/size ratio
    for biscuit in biscuits:
        # Try placing the biscuit in all positions on the roll
        for start_position in range(ROLL_LENGTH):
            while can_place_biscuit(start_position, biscuit, roll):
                # Place the biscuit
                place_biscuit(start_position, biscuit, roll)
                biscuit_counts[biscuit['type']] += 1  # Increment the count for this type of biscuit
                total_value += biscuit['value']  # Add the value of the placed biscuit
                break  # Once the biscuit is placed, move on to the next one

    # Calculate the penalty for unused dough (if any)
    total_penalty = calculate_unused_penalty(roll)

    # Return the total value (including penalty) and the count of each type of biscuit placed
    return total_value, total_value + total_penalty, biscuit_counts


This function implements a greedy strategy for placing biscuits on the roll. Biscuits are sorted by their value-to-size ratio to prioritize high-value placements. It iterates over all positions on the roll, placing biscuits where possible based on defect tolerances. The total value of placed biscuits and their counts are tracked, with a penalty applied for unused dough.


In [11]:
# Initialize the dough roll (0 = no defect, otherwise defect counts for each class)
roll = [{'a': 0, 'b': 0, 'c': 0, 'occupied': False} for _ in range(ROLL_LENGTH)]

# Load defect data from the CSV file
def load_defects(file_path):
    with open(file_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            position = round(float(row['x']))  # Convert 'x' to nearest integer
            defect_class = row['class']        # Read the defect class ('a', 'b', 'c')

            if 0 <= position < ROLL_LENGTH:  # Ensure position is within bounds
                roll[position][defect_class] += 1

# Call the function to load the defects
load_defects('defects.csv')

value, net_value, biscuit_counts = greedy_biscuit_placement(roll, biscuits)

print(f"Value: {value}")
print(f"Net value: {net_value}")
print(f"Biscuit counts: {biscuit_counts}")


Value: 644
Net value: 639
Biscuit counts: {0: 13, 1: 2, 2: 13, 3: 71}




---






**Genetic Algorithm**

The greedy model focuses on local optimization by placing biscuits based on their value-to-size ratio. While effective for quick solutions, it struggles with global optimization, often leaving unused spaces or missing better overall arrangements in rolls with complex defect patterns.


The genetic algorithm addresses the limitations of the greedy approach by using an evolutionary process:  

1. **Population Initialization**: A diverse set of initial solutions is generated to provide a wide range of starting points.  
2. **Selection**: High-performing solutions are chosen based on fitness, ensuring the best candidates influence future generations.  
3. **Crossover**: Solutions are combined to create new ones, mixing traits from parent solutions to explore potential improvements.  
4. **Mutation**: Random changes introduce diversity, preventing the algorithm from getting stuck in local optima.  
5. **Iteration**: Over multiple generations, the algorithm balances exploration and exploitation, refining solutions to approach the global optimum.  

This structured process enables the genetic algorithm to handle complex defect patterns and constraints, often outperforming simpler methods like the greedy approach.

In [12]:
import random
import csv
import copy

# Initialize the dough roll (0 = no defect, otherwise defect counts for each class)
roll_template = [{'a': 0, 'b': 0, 'c': 0, 'occupied': False} for _ in range(ROLL_LENGTH)]

To apply this new algorithm, the dough roll is reset to ensure a clean slate for accurate optimization.








In [13]:
# Function to generate a random placement of biscuits
def generate_random_solution(biscuits, roll):
    solution = []
    current_position = 0
    while current_position < ROLL_LENGTH:
        if not roll[current_position]['occupied']:
            biscuit = random.choice(biscuits)
            if can_place_biscuit(current_position, biscuit, roll):
                solution.append((current_position, biscuit['type']))
                for i in range(biscuit['size']):
                    roll[current_position + i]['occupied'] = True
                current_position += biscuit['size']
            else:
                current_position += 1
        else:
            current_position += 1
    return solution, roll  # Return solution and its associated roll

This function generates a random biscuit placement on the dough roll:  

- It iterates through the roll and randomly selects biscuits to place, checking if they fit without violating constraints.  
- Once a biscuit is placed, the corresponding positions are marked as occupied, and the process moves forward.  
- The function returns the generated solution and the updated roll.  

This approach provides a baseline for initializing solutions in optimization algorithms.

In [14]:
# Function to calculate fitness
def fitness(individual, roll_template, biscuits):
    roll = copy.deepcopy(roll_template)
    total_value = 0
    for position, biscuit_type in individual:
        biscuit = biscuits[biscuit_type]
        total_value += biscuit['value']
        for i in range(biscuit['size']):
            roll[position + i]['occupied'] = True
    return total_value  # Modify this based on penalty logic

The **fitness function** evaluates the quality of a solution by:  

- Creating a copy of the initial roll to simulate biscuit placement.  
- Calculating the total value of biscuits placed according to the solution.  
- Marking the occupied positions on the roll.  

The returned fitness score reflects the solution's effectiveness, which can be extended to include penalties for defects or unused spaces.

In [15]:
# Crossover function
def crossover(parent1, parent2, biscuits, roll_template):
    if random.random() < 0.5:
        parent1, parent2 = parent2, parent1  # Random inversion of parents
    start_positions1 = {pos for pos, _ in parent1[0]}
    start_positions2 = {pos for pos, _ in parent2[0]}
    common_positions = list(start_positions1.intersection(start_positions2))
    if not common_positions:
        return copy.deepcopy(parent1) if fitness(parent1[0], roll_template, biscuits) > fitness(parent2[0], roll_template, biscuits) else copy.deepcopy(parent2)
    crossover_point = random.choice(common_positions)
    child_individual = [gene for gene in parent1[0] if gene[0] < crossover_point] + [gene for gene in parent2[0] if gene[0] >= crossover_point]
    child_roll = copy.deepcopy(roll_template)
    for position in range(len(child_roll)):
        child_roll[position]['occupied'] = False
    for position, biscuit_type in child_individual:
        biscuit = biscuits[biscuit_type]
        for i in range(biscuit['size']):
            child_roll[position + i]['occupied'] = True
    return (child_individual, child_roll)


The **crossover function** combines two parent solutions to create a child solution:

- A random selection determines which parent is used as the base.
- It identifies common starting positions between both parents' biscuit placements.
- If there are no common positions, the function returns the better of the two parents based on fitness.
- If common positions exist, the function performs a crossover by combining the biscuits placed before and after a randomly chosen common position.
- The child’s roll is then updated based on this new arrangement of biscuits.

This approach maintains the diversity of the population while attempting to improve the overall solution.

In [16]:
# Mutation function
def mutate(individual, biscuits, roll_template):
    individual.sort(key=lambda x: x[0])
    roll = copy.deepcopy(roll_template)
    num_mutations = random.randint(5, 100)
    mutation_indices = random.sample(range(len(individual)), min(num_mutations, len(individual)))
    for i in mutation_indices:
        position, biscuit_type = individual[i]
        biscuit_size = biscuits[biscuit_type]['size']
        for j in range(biscuit_size):
            roll[position + j]['occupied'] = False
    individual = [b for i, b in enumerate(individual) if i not in mutation_indices]
    current_position = 0
    while current_position < ROLL_LENGTH:
        if not roll[current_position]['occupied']:
            biscuit = random.choice(biscuits)
            if can_place_biscuit(current_position, biscuit, roll):
                individual.append((current_position, biscuit['type']))
                for i in range(biscuit['size']):
                    roll[current_position + i]['occupied'] = True
                current_position += biscuit['size']
            else:
                current_position += 1
        else:
            current_position += 1
    individual.sort(key=lambda x: x[0])
    return individual, roll


The mutation function randomly alters the current biscuit placement by performing the following steps:

1. **Sorting**: The individual (which represents a solution) is sorted based on the position of biscuits to ensure proper order.
2. **Mutation Selection**: A random number of mutations is determined (between 5 and 100). Random indices are chosen within the individual, and the corresponding biscuits are removed from the roll.
3. **Clearing Occupied Positions**: For each mutated biscuit, the positions it occupies are cleared (set to `False`).
4. **Reinsertion of Biscuits**: The function then attempts to place new biscuits into the cleared positions, following the same placement strategy used earlier in the genetic algorithm.
5. **Sorting Again**: Finally, the individual is re-sorted to maintain the correct position order.

This mutation step introduces diversity by modifying the arrangement of biscuits in the roll, which can help explore different possible solutions in the search space.

In [17]:
# Function to create the next generation
def create_new_generation(population, roll_template, biscuits):
    sorted_population = sorted(population, key=lambda ind: fitness(ind[0], roll_template, biscuits), reverse=True)
    top_10 = sorted_population[:10]
    new_population = []
    while len(new_population) < len(population):
        parent1 = random.choice(top_10)
        parent2 = random.choice(population)
        child, roll_child = crossover(parent1, parent2, biscuits, copy.deepcopy(roll_template))
        child, roll_child = mutate(child, biscuits, roll_child)
        new_population.append((child, roll_child))
        if len(new_population) < len(population):
            new_population.append(parent1)
    return new_population[:len(population)]

The function `create_new_generation` generates the next set of individuals (or solutions) in the genetic algorithm process. Here's a breakdown of how it works:

1. **Sort Population**: The current population is sorted by fitness, with the highest fitness individuals (those with the best solutions) placed at the front.
2. **Select Top 10**: The top 10 individuals from the population are selected based on their fitness. These individuals have the highest chance of contributing to the next generation.
3. **Create New Population**: The function enters a loop, generating new individuals until the new population reaches the size of the original population:
   - **Parent Selection**: Two parents are selected: one from the top 10 (elitism) and one randomly from the entire population.
   - **Crossover**: The parents are used to produce a child (new solution) through crossover.
   - **Mutation**: The child undergoes mutation to introduce diversity.
   - **Add to Population**: The new child is added to the new population. If there is room, a parent is also added to maintain the population size.
4. **Return New Generation**: The new population is returned, ensuring its size is equal to the original population size.

This process promotes the evolution of better solutions over generations by combining the best individuals and introducing mutations for exploration.



---



Here is the final approach for the genetic algorithm applied to the biscuit placement problem. This implementation evolves a population of solutions across multiple generations, aiming to maximize the total value of biscuits placed on the roll.

### Overview of the Process:
1. **Population Initialization**: A randomly generated population of possible solutions is created at the start. Each solution represents a specific arrangement of biscuits on the dough roll.
2. **Selection of Parents**: In each generation, the top 10 solutions are selected based on their fitness (the total value of biscuits placed). These solutions act as parents for the next generation.
3. **Crossover and Mutation**: New solutions are created by combining two parents (crossover) and making slight random changes (mutation) to introduce diversity into the population.
4. **Fitness Evaluation**: The fitness of each solution is evaluated based on the total value of biscuits placed, with higher values indicating better solutions.
5. **Result Monitoring**: At regular intervals (every 10 generations), the fitness of the best solution is displayed, allowing you to monitor how the solution evolves over time.
6. **Final Output**: After running for a predefined number of generations, the best solution and its corresponding fitness score are returned.

### Key Features:
- **Periodic Results Display**: During the evolution process, you can observe the progress by printing the best fitness values at regular intervals (e.g., every 10 generations). This allows you to track how the solution is improving as the generations advance.
- **Optimization Strategy**: The genetic algorithm seeks to refine the placement of biscuits on the roll, exploring different configurations through crossover and mutation to achieve an optimal or near-optimal solution.

By following this approach, the algorithm efficiently optimizes the placement of biscuits, balancing exploration and exploitation to find the most valuable arrangement while reducing the impact of defects on the dough roll.

In [18]:
# Function to run the genetic algorithm
def genetic_algorithm(pop_size, biscuits, roll_template, generations):
    population = [generate_random_solution(biscuits, copy.deepcopy(roll_template)) for _ in range(pop_size)]
    print(f"Generation | Best Fitness: ")
    for generation in range(generations):
        population = create_new_generation(population, copy.deepcopy(roll_template), biscuits)
        if (generation < 100 and generation % 10 == 0) or (generation >= 100 and generation % 100 == 0):
            best_individual = max(population, key=lambda ind: fitness(ind[0], roll_template, biscuits))
            def get_spacing(value):
                if value < 10:
                      return "   "  # Add two spaces for numbers less than 100
                if value < 100:
                    return "  "  # Add two spaces for numbers less than 100
                elif value < 1000:
                    return " "  # Add one space for numbers less than 1000
                return ""  # No space for numbers greater than or equal to 1000
            spacing = get_spacing(generation)
            print(f"      {spacing}{generation} | {fitness(best_individual[0], roll_template, biscuits)}")
    best_solution = max(population, key=lambda ind: fitness(ind[0], roll_template, biscuits))
    return best_solution[0], fitness(best_solution[0], roll_template, biscuits)

# Example usage
best_solution, best_fitness = genetic_algorithm(pop_size=100, biscuits=biscuits, roll_template=roll_template, generations=1000)
print(f"Best solution: {best_solution}\n, Fitness: {best_fitness}")


#Best solution found : [(0, 1), (8, 0), (12, 1), (20, 3), (25, 3), (30, 1), (38, 1), (47, 3), (52, 3), (58, 2), (60, 3), (65, 1), (73, 3), (78, 1), (86, 1), (96, 3), (101, 3), (106, 3), (111, 1), (119, 1), (127, 3), (132, 2), (134, 3), (139, 3), (145, 3), (150, 3), (157, 3), (162, 1), (172, 1), (180, 1), (188, 3), (193, 0), (197, 3), (202, 3), (207, 3), (212, 3), (217, 3), (222, 1), (231, 3), (236, 3), (241, 1), (249, 1), (257, 3), (264, 2), (270, 1), (278, 1), (287, 3), (292, 2), (294, 3), (299, 3), (305, 3), (311, 2), (313, 1), (321, 3), (326, 2), (329, 3), (334, 3), (340, 2), (342, 3), (347, 3), (352, 3), (357, 3), (362, 3), (367, 2), (369, 1), (377, 2), (379, 3), (384, 1), (392, 2), (394, 1), (402, 3), (407, 3), (413, 3), (418, 2), (420, 3), (425, 3), (430, 0), (434, 1), (442, 2), (444, 3), (449, 3), (454, 3), (459, 3), (464, 1), (472, 3), (477, 3), (482, 3), (489, 3), (494, 3)],
#Fitness: 730
#Found with 2000 generation


Generation | Best Fitness: 
         0 | 669
        10 | 680
        20 | 680
        30 | 680
        40 | 696
        50 | 696
        60 | 696
        70 | 696
        80 | 696
        90 | 696
       100 | 696
       200 | 700
       300 | 700
       400 | 700
       500 | 700
       600 | 701
       700 | 701
       800 | 701
       900 | 701
Best solution: [(0, 2), (2, 1), (10, 1), (18, 3), (23, 3), (28, 2), (30, 1), (38, 1), (46, 1), (54, 3), (59, 2), (61, 1), (69, 1), (77, 1), (85, 3), (90, 0), (94, 2), (96, 3), (101, 2), (103, 2), (105, 3), (110, 1), (118, 1), (126, 3), (131, 1), (139, 2), (141, 0), (145, 1), (153, 3), (158, 1), (166, 3), (171, 2), (173, 3), (178, 1), (186, 3), (191, 3), (196, 3), (201, 1), (209, 3), (214, 3), (219, 3), (224, 1), (232, 1), (240, 1), (248, 1), (256, 1), (265, 1), (273, 1), (281, 1), (289, 1), (297, 2), (299, 2), (301, 3), (306, 3), (311, 0), (315, 3), (320, 0), (324, 2), (326, 1), (334, 1), (342, 3), (347, 2), (350, 3), (355, 2), (357, 1), (36

**Results and Performance Analysis**  

The solution performed well in generating viable outcomes for the optimization problem, demonstrating efficiency in terms of convergence rate and solution quality. Execution time was within acceptable limits, but further improvements could optimize performance and enhance the robustness of the solution.  

### Metrics:  
- **Execution Time**: Measured across multiple runs, the algorithm consistently completed within a reasonable timeframe, even with increasing problem complexity.  
- **Solution Quality**: The algorithm successfully avoided most local optima by employing genetic operations like crossover and mutation. The results showed improvement in overall fitness across generations.  

### Strengths:  
1. **Effective Exploration and Exploitation**:  
   The current design leverages genetic diversity and fitness-based selection, ensuring the algorithm explores the solution space effectively.  

2. **Simplicity and Scalability**:  
   The implementation is straightforward and can handle larger datasets or problems with minimal modifications.  

### Suggestions for Improvement:  
1. **Enhanced Mutation and Crossover Randomization**:  
   The current approach fills offspring genes sequentially (left-to-right). Introducing randomness, such as starting at a random position and filling in different directions (clockwise or counterclockwise), can increase genetic diversity.  
   - **Crossover**: Instead of fixed split points, using randomized crossover points and dynamic direction selection for gene filling can lead to more diverse offspring.  
   - **Mutation**: Randomizing mutation operations further, by altering genes at different rates or using a non-linear mutation strategy, could enhance performance.  

2. **Selection Mechanism**:  
   The selection process could incorporate probabilistic elitism or diversity-based selection, where individuals are chosen based on fitness and their uniqueness in the population. This would reduce the risk of premature convergence and maintain variety in the gene pool.  

3. **Diversity-Preservation Techniques**:  
   Implementing mechanisms to measure and preserve population diversity, such as penalizing similar solutions, would further strengthen the algorithm’s exploratory capabilities.  

---

**Final Reflections**  

The project was a valuable learning experience, blending theoretical concepts with practical implementation. Some key takeaways include:  
1. **Challenges Encountered**:  
   - Balancing exploration and exploitation was critical. Initial versions of the algorithm lacked sufficient diversity, leading to suboptimal solutions. This issue highlighted the importance of fine-tuning genetic operations like mutation and crossover.  
   - Execution time, while acceptable, showed room for optimization, particularly with larger datasets.  

2. **What Could Be Done Differently**:  
   - A stronger emphasis on randomization in genetic operations from the beginning would have improved early results. This would have mitigated issues related to convergence and stagnation.  
   - Incorporating additional performance metrics (e.g., population diversity indices) earlier in the project could have provided better insight into algorithm behavior.  

3. **Key Learning Outcomes**:  
   - The interplay between diversity and convergence is crucial in genetic algorithms. Maintaining a balance ensures robust exploration without sacrificing solution quality.  
   - Randomness, when applied strategically, can significantly enhance the performance of heuristic algorithms.  
   - Modular and scalable design ensures the solution can adapt to variations in problem complexity with minimal changes.  

4. **Insights and Observations**:  
   The project reinforced the importance of iterative development. Each adjustment, particularly to the mutation and crossover mechanisms, yielded measurable improvements. Moreover, focusing on randomness as a tool for diversity opened avenues for further experimentation, such as employing adaptive probabilities for genetic operations based on population metrics.  

This project not only provided practical insights into genetic algorithms but also offered a deeper understanding of how randomness and diversity are critical to solving complex optimization problems.